<a href="https://colab.research.google.com/github/Fobbs-DBC-Lab/data-analysis/blob/main/Motivation_PR_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [1]:
import pandas as pd
print('You are running pandas version:', pd.__version__)
import numpy as np
print('You are running numpy version:', np.__version__)
from datetime import datetime as dt
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import glob

pd.set_option('display.max_rows', None)

# This requires Python 3.7 and above

You are running pandas version: 2.2.3
You are running numpy version: 2.0.2


In [ ]:
def read_csv_files(file_path):
    """
    Reads CSV files and organizes them into a single dataframe with file categories
    """
    all_dataframes = []
    file_counter = 1  # Initialize counter for file categories

    # for all files in folder
    for file in os.listdir(file_path):
        if file.endswith(".CSV"):
            if file.startswith("FED"):
                # read that file into a dataframe
                file_path_df = os.path.join(file_path, file)
                df = pd.read_csv(file_path_df)

                # Add file category column
                df['file_num'] = file_counter
                file_counter += 1

                all_dataframes.append(df)

    # create a single dataframe from all files
    singular_df = pd.concat(all_dataframes)
    pd.to_datetime(singular_df['MM:DD:YYYY hh:mm:ss'])
    #singular_df = singular_df.sort_values(by=['MM:DD:YYYY hh:mm:ss'], ascending=True)
    return singular_df

In [ ]:
#If Wambura is running file - change mouse & FED number accordingly
MOTH_PATH ='/content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/Chow Group/'
MOUSE_PATH = '/content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/Chow Group/m281_FED004/'
Analysis_Files = '/content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/SD Analyses/'
MOUSE = 'M281'
FED = '_FED004'
DATE = '022325'

#Keep for both Wambura and Kevin runs
print ("Number of files found: ")
print(len(os.listdir(MOUSE_PATH)))

Number of files found: 
39


## Kevin's Path

In [ ]:
# # If Kevin is running file - change mouse & FED number accordingly
KEV_MOUSE_PATH = '/content/drive/MyDrive/Motivation Project 2023/M18/'
MOT_PATH = '/content/drive/MyDrive/Motivation Project 2023/'

MOUSE = 'M18'
FED = '_FED022'
DATE = '11032024'

#Keep for both Wambura and Kevin runs
print ("Number of files found: ")
print(len(os.listdir(MOT_PATH)))

Number of files found: 


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Motivation Project 2023/'

## Processing FED Device Data Files

This script processes FED (Feeding Experimentation Device) data files by adding two new columns that track the beginning and end dates for each file number in the dataset.

### Input Data Structure
The script expects a CSV file with the following key columns:
- An unnamed index column (first column)
- `MM:DD:YYYY hh:mm:ss`: Timestamp column in the format "M/D/YYYY HH:MM:SS"
- `file_num`: Integer column indicating the file number

### New Columns Added
The script adds two new columns to the dataset:
- `file_begin_date`: The date (MM/DD/YYYY) when each file_num first appears
- `file_end_date`: The date (MM/DD/YYYY) when each file_num last appears

### Processing Steps
1. Reads the CSV file using pandas, setting the unnamed first column as the index
2. Converts the "MM:DD:YYYY hh:mm:ss" column to datetime format for proper date handling
3. Groups the data by file_num to identify:
   - First timestamp for each file number (begin_date)
   - Last timestamp for each file number (end_date)
4. Creates new columns mapping these dates back to the original dataframe
5. Formats dates in MM/DD/YYYY format
6. Saves the processed data to a new CSV file

### Usage
```python
input_file = "SD Analyses M281 FED004.csv"
output_file = "SD Analyses M281 FED004_processed.csv"
processed_df = process_fed_dates(input_file, output_file)
```

### Output
The script creates a new CSV file with all original columns plus:
- file_begin_date
- file_end_date

It also prints:
- A sample of the processed data (first 5 rows)
- A summary showing the date ranges for each file number

Note: The original timestamp format is preserved in the "MM:DD:YYYY hh:mm:ss" column, while the new date columns are formatted as MM/DD/YYYY for clarity.

##Dict

In [ ]:
def process_fed_dates(input_file, output_file):
    """
    Process FED device CSV file to add file_begin_date and file_end_date columns.

    Parameters:
    input_file (str): Path to input CSV file
    output_file (str): Path to save the processed CSV file
    """
    # Read the CSV file
    # Note: the first column is unnamed and just an index, so we'll use it as the index
    df = pd.read_csv(input_file, index_col=0)

    # Convert datetime column to datetime type
    df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'])

    # Group by file_num and get first and last dates
    file_dates = df.groupby('file_num').agg({
        'MM:DD:YYYY hh:mm:ss': ['first', 'last']
    })

    # Rename columns for clarity
    file_dates.columns = ['begin_date', 'end_date']

    # Convert to dictionary for mapping
    begin_dates = file_dates['begin_date'].to_dict()
    end_dates = file_dates['end_date'].to_dict()

    # Add new columns to original dataframe
    df['file_begin_date'] = df['file_num'].map(begin_dates).dt.strftime('%m/%d/%Y')
    df['file_end_date'] = df['file_num'].map(end_dates).dt.strftime('%m/%d/%Y')

    # Save to new CSV file
    df.to_csv(output_file)

    return df

## Example usage

In [ ]:
# Example usage:
if __name__ == "__main__":
    input_file = "SD Analyses M281 FED004.csv"
    output_file = "SD Analyses M281 FED004_processed.csv"

    processed_df = process_fed_dates(input_file, output_file)
    print("Processing complete. New file saved as:", output_file)

    # Display sample of processed data
    print("\nSample of processed data (first 5 rows):")
    print(processed_df[['file_num', 'MM:DD:YYYY hh:mm:ss', 'file_begin_date', 'file_end_date']].head())

    # Print summary of file numbers and their date ranges
    summary = processed_df.groupby('file_num').agg({
        'file_begin_date': 'first',
        'file_end_date': 'first'
    })
    print("\nSummary of file number date ranges:")
    print(summary.head())

## Read CSV Files + concat 1 (append file number) + concat 2 (add date at file end)

In [ ]:
concat_df = read_csv_files(MOUSE_PATH) # Point to the specific FED folder for Wambura (i.e., 1, 2, 3 etc)
concat_df.to_csv(Analysis_Files + MOUSE + FED + '_concat1_' + DATE + '.csv')
concat_df.head(10)

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,file_num
0,3/27/2024 13:12:18,1.14.0,ClosedEcon,4,4.07,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.13,1
1,3/27/2024 13:18:49,1.14.0,ClosedEcon,4,4.07,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.14,1
2,3/27/2024 13:19:03,1.14.0,ClosedEcon,4,4.07,4.0,1,Pellet,Left,1,1,1,1,8.84,NaN,NaN,1
3,3/27/2024 13:20:00,1.14.0,ClosedEcon,4,4.07,NaN,2,Left,Left,2,1,1,1,NaN,NaN,0.18,1
4,3/27/2024 13:20:10,1.14.0,ClosedEcon,4,4.07,NaN,2,Right,Left,2,2,1,1,NaN,NaN,0.03,1
5,3/27/2024 13:20:20,1.14.0,ClosedEcon,4,4.07,NaN,2,Left,Left,3,2,1,1,NaN,NaN,0.04,1
6,3/27/2024 13:20:50,1.14.0,ClosedEcon,4,4.07,2.0,2,Pellet,Left,3,2,2,2,28.51,106.0,NaN,1
7,3/27/2024 13:25:32,1.14.0,ClosedEcon,4,4.07,NaN,3,Left,Left,4,2,2,2,NaN,NaN,0.06,1
8,3/27/2024 13:26:34,1.14.0,ClosedEcon,4,4.07,NaN,3,Left,Left,5,2,2,2,NaN,NaN,0.21,1
9,3/27/2024 13:26:40,1.14.0,ClosedEcon,4,4.07,NaN,3,Right,Left,5,3,2,2,NaN,NaN,0.03,1


In [ ]:
if __name__ == "__main__":
    input_file = '/content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/SD Analyses/M281_FED004_concat1_022325.csv'
    output_file = '/content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/SD Analyses/M281_FED004_concat2_022325.csv'

    concat2_df = process_fed_dates(input_file, output_file)
    print("Processing complete. New file saved as:", output_file)

    # Display sample of processed data
    print("\nSample of processed data (first 5 rows):")
    print(processed_df[['file_num', 'MM:DD:YYYY hh:mm:ss', 'file_begin_date', 'file_end_date']].head())

    # Print summary of file numbers and their date ranges
    summary = processed_df.groupby('file_num').agg({
        'file_begin_date': 'first',
        'file_end_date': 'first'
    })
    print("\nSummary of file number date ranges:")
    print(summary.head())

Processing complete. New file saved as: /content/drive/Shared drives/[PSYC] DBC Lab/Project Files/Motivation Project 2024/PR/SD Data/SD Analyses/M281_FED004_concat2_022325.csv

Sample of processed data (first 5 rows):
   file_num MM:DD:YYYY hh:mm:ss file_begin_date file_end_date
0         1 2024-03-25 20:07:20      03/25/2024    03/26/2024
1         1 2024-03-25 20:09:25      03/25/2024    03/26/2024
2         1 2024-03-25 20:09:52      03/25/2024    03/26/2024
3         1 2024-03-25 20:10:54      03/25/2024    03/26/2024
4         1 2024-03-25 20:10:57      03/25/2024    03/26/2024

Summary of file number date ranges:
         file_begin_date file_end_date
file_num                              
1             03/25/2024    03/26/2024
2             03/26/2024    03/27/2024
3             03/27/2024    03/28/2024
4             03/28/2024    03/29/2024
5             03/29/2024    03/30/2024


## Compute Total Values Per day (File number)

In [ ]:
concat2_df.loc[(concat2_df['Event'] == 'Right') | (concat2_df['Event'] == 'Left') | (concat2_df['Event'] == 'Pellet') | (concat2_df['Event']=='LeftWithPellet') | (concat2_df['Event']=='RightWithPellet'), 'Event Binary'] = 1
concat2_df['meal_number'] = (
    concat2_df
    .groupby('file_num')  # Group by file number first
    ['FR']
    .diff()              # Calculate differences within each file
    .lt(0)              # Find where FR decreases
    .cumsum()           # Cumulative sum to get meal numbers
    + 1                 # Add 1 to start meals at 1 instead of 0
)
concat2_df.to_csv(Analysis_Files + MOUSE + FED + '_concat3test_' + DATE + '.csv')

In [ ]:
concat3_df = pd.DataFrame(concat2_df[['file_num', 'Event', 'Event Binary']].groupby(['file_num','Event']).sum())
concat3_df.head()

Event Binary
file_num Event                       
1        Left                  2855.0
         LeftWithPellet          15.0
         Pellet                 175.0
         Right                  141.0
2        Left                  1904.0

In [ ]:
daily_df = concat3_df.groupby(['file_num','Event'])['Event Binary'].first().unstack(fill_value=1).rename_axis(None)
daily_df['Pokes Per Pellet (Left/Pellet)'] = daily_df['Left']/daily_df['Pellet']
daily_df['Pellet (kcal)'] = daily_df['Pellet']*0.066
daily_df['Left Total'] = daily_df['Left']+daily_df['LeftWithPellet']
daily_df['Right Total'] = daily_df['Right']+ daily_df['RightWithPellet']
daily_df['Total Pokes'] = daily_df['Left Total'] + daily_df['Right Total']
daily_df['%Correct (All)'] = daily_df['Left Total']/daily_df['Total Pokes']
daily_df['%Correct (no pellet)'] = daily_df['Left']/ (daily_df['Left']+daily_df['Right'])
daily_df['Max FR reached'] = concat2_df.groupby('file_num')['FR'].max() #--> this is more of a max reached FR rather than a max completed
#daily_df['Mean FR'] = concat2_df.groupby('file_num')['FR'].mean() # add this line to production <- this calculation is NOT what we want because it averages across every event line
daily_df['Mean Inter-pellet Interval']= concat2_df.groupby('file_num')['InterPelletInterval'].mean() # add this line to production (10/22/24)
daily_df['Max Inter-pellet Interval']= concat2_df.groupby('file_num')['InterPelletInterval'].max() # add this line to production (10/22/24)
daily_df['Min Inter-pellet Interval']= concat2_df.groupby('file_num')['InterPelletInterval'].min() # add this line to production (10/22/24)
#daily_df['Mode Inter-pellet Interval']= concat2_df.groupby('file_num').agg({'InterPelletInterval':pd.Series.mode}) # add this line to production (10/22/24)
#daily_df['Number of Meals'] = concat2_df.groupby('file_num').agg({'meal_number': pd.Series.nunique}) # add this line to production if you don't already have it
daily_df = daily_df.reset_index()
#daily_df = tdf.rename(columns={'index': 'file_num'})

#Wambura
daily_df.to_csv(Analysis_Files + MOUSE + FED + '_binned_by_file_' + DATE + '.csv')
daily_df.head(100)


Event,index,Left,LeftWithPellet,Pellet,Right,RightWithPellet,Pokes Per Pellet (Left/Pellet),Pellet (kcal),Left Total,Right Total,Total Pokes,%Correct (All),%Correct (no pellet),Max FR reached,Mean Inter-pellet Interval,Max Inter-pellet Interval,Min Inter-pellet Interval
0,1,2855.0,15.0,175.0,141.0,1.0,16.314286,11.550,2870.0,142.0,3012.0,0.952855,0.952937,56,461.896552,6800.0,22.0
1,2,1904.0,1.0,190.0,38.0,1.0,10.021053,12.540,1905.0,39.0,1944.0,0.979938,0.980433,37,435.724868,6962.0,29.0
2,3,1438.0,1.0,172.0,62.0,1.0,8.360465,11.352,1439.0,63.0,1502.0,0.958056,0.958667,31,495.567251,5002.0,26.0
3,4,1278.0,3.0,186.0,24.0,1.0,6.870968,12.276,1281.0,25.0,1306.0,0.980858,0.981567,29,461.254054,4609.0,24.0
4,5,1219.0,1.0,170.0,19.0,1.0,7.170588,11.220,1220.0,20.0,1240.0,0.983871,0.984653,30,500.757396,4920.0,25.0
5,6,1114.0,5.0,167.0,20.0,1.0,6.670659,11.022,1119.0,21.0,1140.0,0.981579,0.982363,27,507.957831,5057.0,22.0
6,7,1003.0,1.0,150.0,16.0,1.0,6.686667,9.900,1004.0,17.0,1021.0,0.983350,0.984298,28,558.335570,5075.0,21.0
7,8,948.0,1.0,148.0,35.0,1.0,6.405405,9.768,949.0,36.0,985.0,0.963452,0.964395,29,593.585034,6499.0,25.0
8,9,1312.0,1.0,151.0,21.0,1.0,8.688742,9.966,1313.0,22.0,1335.0,0.983521,0.984246,38,567.673333,6479.0,19.0
9,10,1081.0,1.0,161.0,65.0,1.0,6.714286,10.626,1082.0,66.0,1148.0,0.942509,0.943281,29,529.093750,5133.0,19.0


## Plotting Practice

In [ ]:
concat_df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(concat_df['MM:DD:YYYY hh:mm:ss'])
concat_df["Time(hh:mm:ss)"] = concat_df["MM:DD:YYYY hh:mm:ss"].dt.strftime("%H:%M:%S")
concat_df.head(10)

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,file_num,Time(hh:mm:ss)
0,2024-03-25 20:07:20,1.14.0,Free_feed,4,4.20,4.0,1,Pellet,Left,0,0,1,0,Timed_out,NaN,NaN,1,20:07:20
1,2024-03-25 20:09:25,1.14.0,Free_feed,4,4.20,NaN,1,LeftWithPellet,Left,1,0,1,0,NaN,NaN,0.62,1,20:09:25
2,2024-03-25 20:09:52,1.14.0,Free_feed,4,4.19,NaN,1,RightWithPellet,Left,1,1,1,0,NaN,NaN,0.10,1,20:09:52
3,2024-03-25 20:10:54,1.14.0,Free_feed,4,4.20,NaN,1,RightWithPellet,Left,1,2,1,0,NaN,NaN,0.15,1,20:10:54
4,2024-03-25 20:10:57,1.14.0,Free_feed,4,4.19,NaN,1,LeftWithPellet,Left,2,2,1,0,NaN,NaN,0.33,1,20:10:57
5,2024-03-25 20:11:55,1.14.0,Free_feed,4,4.18,NaN,1,RightWithPellet,Left,2,3,1,0,NaN,NaN,0.33,1,20:11:55
6,2024-03-25 20:12:55,1.14.0,Free_feed,4,4.20,1.0,1,Pellet,Left,2,3,2,0,Timed_out,335.0,NaN,1,20:12:55
7,2024-03-25 20:13:32,1.14.0,Free_feed,4,4.19,NaN,1,RightWithPellet,Left,2,4,2,0,NaN,NaN,0.17,1,20:13:32
8,2024-03-25 20:14:57,1.14.0,Free_feed,4,4.19,2.0,1,Pellet,Left,2,4,3,0,Timed_out,122.0,NaN,1,20:14:57
9,2024-03-25 20:15:08,1.14.0,Free_feed,4,4.19,2.0,1,Pellet,Left,2,4,4,0,4.10,11.0,NaN,1,20:15:08


In [ ]:
sns_plot, ax = plt.subplots(figsize=(30, 15))
sns.set_style("white")
#plt.ylim(0,100)
#plt.axhline(y=20, color='green', linestyle='--')
#plt.xticks(ticks=[],labels=[])
#data = concat_df[concat_df['file_num']==1]
sns_plot = sns.lineplot(concat_df, x= "Time(hh:mm:ss)", y="Pellet_Count", hue="file_num", palette= 'rocket') #hue="Mouse" ci= 68



Error in callback <function flush_figures at 0x7e3f76fae660> (for post_execute):


KeyboardInterrupt: 